In [92]:
import random
import numpy as np
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from vectorstore import connect_to_db
from openai import OpenAI
import json 

# Historique d'utilisation des clusters (pour la pénalisation)
cluster_usage = defaultdict(int)

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"  # Replace with your actual API key
)

model = "mistralai/mistral-7b-instruct:free"
QUESTIONS_PER_THEME = 3  # Number of questions per theme

In [ ]:
# Récupérer les chunks de la base de données avec leurs embeddings
def fetch_chunks_from_weaviate():
    client = connect_to_db()

    # Accéder à la collection 'Chunk'
    collection = client.collections.get("Chunk")
    
    # Récupérer tous les objets de la collection avec leurs vecteurs
    objects = collection.query.fetch_objects(
        include_vector=True  # Inclut le vecteur dans les résultats
    )
    
    chunks = []
    for obj in objects.objects:
        # Construire le dictionnaire pour chaque chunk
        chunk = obj.properties
        chunk['vector'] = np.array(obj.vector)
        chunks.append(chunk)
        
    print(len(chunks))

    return chunks

In [94]:
# Attribuer les sous-thèmes à chaque chunk
def assign_subthemes_to_chunks(chunks, themes, embedder):
    for chunk in chunks:
        chunk_embedding = chunk['vector']
        assigned_subthemes = []

        for theme, subthemes in themes.items():
            for subtheme, description in subthemes.items():
                subtheme_embedding = embedder.encode(subtheme)  # Encoder le sous-thème
                description_embedding = embedder.encode(description)

                similarity_subtheme = np.dot(chunk_embedding, subtheme_embedding)
                similarity_desc = np.dot(chunk_embedding, description_embedding)

                # Pondération : accorder plus d'importance au sous-thème qu'à la description
                similarity = 0.8 * similarity_subtheme + 0.2 * similarity_desc

                if similarity > 0.8:  # Seuil d'attribution
                    assigned_subthemes.append(subtheme)

        chunk["subthemes"] = assigned_subthemes if assigned_subthemes else ["Unknown"]

    return chunks

# Clustering des chunks avec K-Means
def cluster_chunks(chunks, n_clusters=10):
    """ Crée des clusters de chunks par K-Means """
    if len(chunks) < n_clusters:
        return {i: [chunk] for i, chunk in enumerate(chunks)}
    
    embeddings = np.array([chunk['vector'] for chunk in chunks])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)
    
    clustered_chunks = defaultdict(list)
    for i, label in enumerate(labels):
        clustered_chunks[label].append(chunks[i])
    
    return clustered_chunks

# Sélection du cluster avec pénalité d'utilisation récente et bruit aléatoire
cluster_usage = defaultdict(int)  # Stocke le nombre d'utilisations des clusters

def select_cluster_with_penalty(clustered_chunks):
    cluster_scores = {}

    for cluster_id, chunks in clustered_chunks.items():
        usage_penalty = cluster_usage[cluster_id] * 0.05  # Plus utilisé = plus pénalisé
        cluster_scores[cluster_id] = len(chunks) - usage_penalty
    
    # Ajouter du bruit aléatoire
    for cluster_id in cluster_scores:
        cluster_scores[cluster_id] += random.uniform(-0.1, 0.1)
    
    selected_cluster = max(cluster_scores, key=cluster_scores.get)
    cluster_usage[selected_cluster] += 1  # Mise à jour de la pénalité

    return selected_cluster

# Sélectionner aléatoirement x chunks dans un cluster
def select_chunks_from_cluster(clustered_chunks, selected_cluster, num_chunks=3):
    cluster_chunks = clustered_chunks[selected_cluster]
    return random.sample(cluster_chunks, min(num_chunks, len(cluster_chunks)))

In [95]:
def generate_questions_from_chunks(chunks, model, selected_subtheme):
    """Génère des questions MCQ à partir des chunks sélectionnés en utilisant un LLM."""
    
    combined_texts = "\n\n".join([f"- {chunk['text']}" for chunk in chunks])
    theme = chunks[0].get("themes", ["General"])[0]  # Utiliser le premier thème trouvé

        # Collecter les informations source (original_filename, title, section)
     # Collecter les informations source (original_filename, title, section) sous forme de dictionnaire
    sources = {
        "original_filename": chunks[0].get("original_filename", ""),
        "title": chunks[0].get("title", ""),
        "section": chunks[0].get("section", "")
    }
    
    messages = [
        {"role": "system", "content": "You are an AI that generates multiple-choice questions (MCQs)."},
        {"role": "user", "content": f"""
         
    Based on this theme:
            
    {selected_subtheme}

    Based on the following context:

    {combined_texts}

    Generate {QUESTIONS_PER_THEME} MCQ questions related to this content.
    Each question must strictly follow this JSON format:

    [
        {{
            "QuestionText": "Question text",
            "Options": {{
                "A": "Option A",
                "B": "Option B",
                "C": "Option C",
                "D": "Option D"
            }},
            "CorrectAnswer": "A",  # One letter corresponding to the correct answer
            "Explanation": "Explanation of the correct answer"
        }},
        ...
    ]

    Ensure the output is a **valid JSON array** and contains **no additional text**. Also, make sure the **explanation is well detailed**.
        """}
    ]

    # Appel à l'API OpenAI ou autre LLM
    response = client.chat.completions.create(model=model, messages=messages)

    if response.choices:
        try:
            generated_text = response.choices[0].message.content.strip()
            questions_json = json.loads(generated_text)  # Convertir en JSON
        except json.JSONDecodeError:
            print(f"🚨 JSON error with theme {theme}: {generated_text}")
            return []

    # Formater les questions
    formatted_questions = []
    for i, question in enumerate(questions_json):
        formatted_questions.append({
            "QuestionNumber": i + 1,
            "QuestionText": question.get("QuestionText", "Question not found"),
            "Options": question.get("Options", {}),
            "CorrectAnswer": question.get("CorrectAnswer", ""),
            "Explanation": question.get("Explanation", ""),
            "Question_type": "MCQ",
            "file": "Generated Questions",
            "Theme": selected_subtheme,
            "Sources": sources
        })
   
    return formatted_questions

In [ ]:
chunks = fetch_chunks_from_weaviate()


# Regrouper les chunks par sous-thème
chunks_by_subtheme = defaultdict(list)
for chunk in chunks:
    if "theme" not in chunk or not chunk["theme"]:
        continue  # Ignore ce chunk

    for subtheme in chunk["theme"]:
        chunks_by_subtheme[subtheme].append(chunk)



# Sélectionner un sous-thème aléatoire
selected_subtheme = random.choice(list(chunks_by_subtheme.keys()))

    # Clustering des chunks de ce sous-thème
clusters = cluster_chunks(chunks_by_subtheme[selected_subtheme])

    # Sélectionner un cluster
selected_cluster = select_cluster_with_penalty(clusters)

    # Prendre x chunks dans ce cluster
selected_chunks = select_chunks_from_cluster(clusters, selected_cluster, num_chunks=10)


question = generate_questions_from_chunks(selected_chunks, model, selected_subtheme)


# Nom du fichier dans lequel vous voulez stocker les données JSON
file_name = 'questions.json'

# Sauvegarder les questions générées dans un fichier JSON
with open(file_name, 'w') as json_file:
    json.dump(question, json_file, indent=4)

#print("Thème sélectionné :", selected_subtheme)
#print("Question générée :", question)

10


/usr/lib/python3.9/asyncio/selector_events.py:702: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=84 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/usr/lib/python3.9/asyncio/selector_events.py:702: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=80 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
/usr/lib/python3.9/asyncio/selector_events.py:702: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=86 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


KeyboardInterrupt: 

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import json



def ajouter_questions_firestore(chemin_fichier_json, nom_collection):
    """
    Lit les données d'un fichier JSON et les ajoute à une collection Firestore.

    Args:
        chemin_fichier_json (str): Chemin vers le fichier JSON contenant les données.
        nom_collection (str): Nom de la collection Firestore.
    """
        # Vérifier si Firebase est déjà initialisé
    if not firebase_admin._apps:
        cred = credentials.Certificate("data0battle-firebase-adminsdk-fbsvc-2fa20219b9.json")
        firebase_admin.initialize_app(cred)

    # Connexion à Firestore
    db = firestore.client()
    
    try:
        with open(chemin_fichier_json, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Vérifie que le JSON est bien une liste de dictionnaires
        if not isinstance(data, list):
            raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

        collection_ref = db.collection(nom_collection)

        for question in data:
            if not isinstance(question, dict):
                print(f"⚠️ Erreur : Un élément du JSON n'est pas un dictionnaire : {question}")
                continue  # Ignore les mauvais formats
            
            # Ajoute chaque question comme un document unique dans Firestore
            doc_ref = collection_ref.document(f"{question['QuestionNumber']}_{question['file']}")  # Utilisation du numéro de question comme ID
            doc_ref.set(question)

        print(f"✅ Données ajoutées avec succès à Firestore dans '{nom_collection}'.")

    except FileNotFoundError:
        print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' est introuvable.")
    except json.JSONDecodeError:
        print(f"❌ Erreur : Le fichier '{chemin_fichier_json}' n'est pas un JSON valide.")
    except ValueError as ve:
        print(f"❌ Erreur de format JSON : {ve}")
    except Exception as e:
        print(f"❌ Une erreur s'est produite : {e}")

# Exemple d'utilisation
chemin_fichier_json = 'questions.json'
nom_collection = 'rag'

ajouter_questions_firestore(chemin_fichier_json, nom_collection)

✅ Données ajoutées avec succès à Firestore dans 'rag'.
